<a href="https://colab.research.google.com/github/Kavipriya01/Battle-Neighborhood/blob/Temp_codes/Segmenting_and_clustering_neighborhood_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

Converted addresses into their equivalent latitude and longitude values. Also, Used the **Foursquare API** to explore neighborhoods in Toronto City.Used the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. Used use the **_k_-means clustering** algorithm to complete this task. Finally,used the **Folium library** to visualize the neighborhoods in Toronto City and their emerging clusters.

Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 



In [21]:
import pandas as pd
import numpy as np
!pip install bs4
!pip install requests

from bs4 import BeautifulSoup
import requests
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page=requests.get(url)
page=page.text

soup=BeautifulSoup(page,"html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"7f8e09f8-1ea2-44af-9326-bb6785f54c29","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1019189119,"wgRevisionId":1019189119,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia

In [23]:
##created a dataframe

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [25]:
print(df.shape)
df

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [26]:
!pip install geocoder

import geocoder # import geocoder
latitude=[]
longitude=[]
for index,row in df.iterrows():
    postal_code=row["PostalCode"]
    print(postal_code)
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
print(latitude)
print(longitude)


## Adding latitude and longitude in a df
df['Latitude']=latitude
df['Longitude']=longitude
df

     |████████████████████████████████| 102kB 6.1MB/s 
M3A
[43.75245000000007, -79.32990999999998]
M4A
[43.73057000000006, -79.31305999999995]
M5A
[43.65512000000007, -79.36263999999994]
M6A
[43.72327000000007, -79.45041999999995]
M7A
[43.66253000000006, -79.39187999999996]
M9A
[43.662630000000036, -79.52830999999998]
M1B
[43.811390000000074, -79.19661999999994]
M3B
[43.74923000000007, -79.36185999999998]
M4B
[43.70718000000005, -79.31191999999999]
M5B
[43.65739000000008, -79.37803999999994]
M6B
[43.70687000000004, -79.44811999999996]
M9B
[43.65034000000003, -79.55361999999997]
M1C
[43.78574000000003, -79.15874999999994]
M3C
[43.72168000000005, -79.34351999999996]
M4C
[43.68970000000007, -79.30681999999996]
M5C
[43.65215000000006, -79.37586999999996]
M6C
[43.69211000000007, -79.43035999999995]
M9C
[43.64857000000006, -79.57824999999997]
M1E
[43.765750000000025, -79.17469999999997]
M4E
[43.67709000000008, -79.29546999999997]
M5E
[43.64536000000004, -79.37305999999995]
M6E
[43.6878400000

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
5,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills North,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


In [27]:
print(df.shape)
print(df.columns)

(103, 5)
Index(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')


#### explore the data



We need only Borough,Neighborhood,Latitude,Longitude columns for our clusetering .

So, create a new data frame 

In [34]:
# instantiate new dataframe
data=[df['Borough'],df['Neighborhood'],df['Latitude'],df['Longitude']]
columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude']

neighborhoods = pd.concat(data, axis=1, keys=columns)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.75245,-79.32991
1,North York,Victoria Village,43.73057,-79.31306
2,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [36]:
neighborhoods.shape # shape of the df 


(103, 4)

In [35]:
## make sure the df has 5 boroughs and 306 neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City.


In [87]:
address = 'Toronto City'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [88]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.


In [44]:
neighborhoods['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East York/East Toronto', 'Central Toronto', 'Mississauga',
       'Downtown Toronto Stn A', 'Etobicoke Northwest',
       'East Toronto Business'], dtype=object)

Decided to work with only boroughs that contain the word Toronto 

In [61]:
manhattan_data = neighborhoods[(neighborhoods['Borough'] == 'Downtown Toronto')| (neighborhoods['Borough'] == 'East Toronto') |
                              (neighborhoods['Borough'] == 'West Toronto') | (neighborhoods['Borough'] == 'East York/East Toronto') |
                              (neighborhoods['Borough'] == 'Central Toronto') | (neighborhoods['Borough'] == 'Downtown Toronto Stn A') |
                              (neighborhoods['Borough'] == 'East Toronto Business')].reset_index(drop=True)                
manhattan_data.shape

(39, 4)

In [89]:
## geographical coordinates of manhatten data

address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [90]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan


utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Foursquare Credentials and Version


In [66]:
CLIENT_ID = 'F5PTHIVYMRU0KTRGJ15R43UGC2YQPPOTY2M5TVVKNCMFDG2A' # your Foursquare ID
CLIENT_SECRET = 'FQXS2CGW0YHTWSX55Z0HZD2VJWALC1OM1DEZXEXRMIDQG1VA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F5PTHIVYMRU0KTRGJ15R43UGC2YQPPOTY2M5TVVKNCMFDG2A
CLIENT_SECRET:FQXS2CGW0YHTWSX55Z0HZD2VJWALC1OM1DEZXEXRMIDQG1VA


Get the neighborhood's name,latitude and longitude values.


In [67]:
manhattan_data.loc[0, 'Neighborhood']

neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65512000000007, -79.36263999999994.


#### the top 100 venues that are in Marble Hill within a radius of 500 meters.


In [68]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
print(url) # display URL

## examine the results
results = requests.get(url).json()
print(results)

https://api.foursquare.com/v2/venues/explore?&client_id=F5PTHIVYMRU0KTRGJ15R43UGC2YQPPOTY2M5TVVKNCMFDG2A&client_secret=FQXS2CGW0YHTWSX55Z0HZD2VJWALC1OM1DEZXEXRMIDQG1VA&v=20180605&ll=43.65512000000007,-79.36263999999994&radius=500&limit=100
{'meta': {'code': 200, 'requestId': '608a531e56e0e270054f1308'}, 'response': {'headerLocation': 'Corktown', 'headerFullLocation': 'Corktown, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 23, 'suggestedBounds': {'ne': {'lat': 43.65962000450007, 'lng': -79.3564319112327}, 'sw': {'lat': 43.650619995500065, 'lng': -79.36884808876718}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '54ea41ad498e9a11e9e13308', 'name': 'Roselle Desserts', 'location': {'address': '362 King St E', 'crossStreet': 'Trinity St', 'lat': 43.653446723052674, 'lng': -79.362016717438

In [69]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
### json to dataframe


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head())

## venues returned from foursquareAPI

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

                     name      categories        lat        lng
0        Roselle Desserts          Bakery  43.653447 -79.362017
1           Tandem Coffee     Coffee Shop  43.653559 -79.361809
2  Figs Breakfast & Lunch  Breakfast Spot  43.655675 -79.364503
3         The Yoga Lounge     Yoga Studio  43.655515 -79.364955
4     Body Blitz Spa East             Spa  43.654735 -79.359874
23 venues were returned by Foursquare.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


## 2. Explore Neighborhoods in Manhattan


In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [72]:
## size of a dataframe
print(manhattan_venues.shape)
print(manhattan_venues.head())

## venues for each neighborhood
manhattan_venues.groupby('Neighborhood').count()

## unique categories
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

(1698, 7)
                Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65512               -79.36264   
1  Regent Park, Harbourfront               43.65512               -79.36264   
2  Regent Park, Harbourfront               43.65512               -79.36264   
3  Regent Park, Harbourfront               43.65512               -79.36264   
4  Regent Park, Harbourfront               43.65512               -79.36264   

                    Venue  Venue Latitude  Venue Longitude  Venue Category  
0        Roselle Desserts       43.653447       -79.362017          Bakery  
1           Tandem Coffee       43.653559       -79.361809     Coffee Shop  
2  Figs Breakfast & Lunch       43.655675       -79.364503  Breakfast Spot  
3         The Yoga Lounge       43.655515       -79.364955     Yoga Studio  
4     Body Blitz Spa East       43.654735       -79.359874             Spa  
There are 228 uniques categories.


## Analyze Each Neighborhood


In [73]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

print(manhattan_onehot.head())

print("size of a dataframe")
print(manhattan_onehot.shape)

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
print(manhattan_grouped)

print("new size")
print(manhattan_grouped.shape)

   Yoga Studio  Accessories Store  Adult Boutique  American Restaurant  \
0            0                  0               0                    0   
1            0                  0               0                    0   
2            0                  0               0                    0   
3            1                  0               0                    0   
4            0                  0               0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  BBQ Joint  Baby Store  Bagel Shop  \
0                 0       

In [74]:
## each neighborhood alone with top 5 common venues

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.06
2              Bakery  0.05
3  Seafood Restaurant  0.05
4          Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.06
1             Bar  0.06
2     Coffee Shop  0.05
3      Restaurant  0.05
4  Sandwich Place  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.06
2                Café  0.05
3                Park  0.04
4                 Bar  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.11
1             Clothing Store  0.10
2             Sandwich Place  0.05
3  Middle Eastern Restaurant  0.03
4                Pizza Place  0.03


----Christie----
                venue  freq
0                Café  0.27
1       Grocer

In [75]:
## function to sort venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:

## displaying top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Restaurant,Coffee Shop,Gift Shop,Sandwich Place,Lounge,Italian Restaurant,Breakfast Spot,Nightclub
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Café,French Restaurant,Sandwich Place,Bar,Park,Gym / Fitness Center,Intersection,Grocery Store
3,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Pizza Place,Plaza,Bubble Tea Shop,Cosmetics Shop,Furniture / Home Store,Bookstore
4,Christie,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Candy Store,Playground,Event Space,Dumpling Restaurant


## Cluster Neighborhoods


In [77]:
## k-means clustering
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [78]:
## new dataframe with all top 10 venues
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Wine Shop,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Ramen Restaurant
2,Downtown Toronto,St. James Town,43.65215,-79.37587,1,Coffee Shop,Cosmetics Shop,Hotel,Clothing Store,Cocktail Bar,Italian Restaurant,Café,Seafood Restaurant,American Restaurant,Restaurant
3,East Toronto,The Beaches,43.67709,-79.29547,1,Health Food Store,Trail,Pub,Asian Restaurant,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,Downtown Toronto,Berczy Park,43.64536,-79.37306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Breakfast Spot


In [79]:
## visualize clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories,assign a name to each cluster.



In [80]:
## cluster 1

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,The Danforth East,Convenience Store,Park,Intersection,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
12,"The Danforth West, Riverdale",Business Service,Ice Cream Shop,Grocery Store,Bus Line,Park,Discount Store,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
22,"High Park, The Junction South",Sandwich Place,Park,Residential Building (Apartment / Condo),Wings Joint,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
23,North Toronto West,Gym Pool,Playground,Park,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
33,Rosedale,Park,Japanese Restaurant,Playground,Bike Trail,Wings Joint,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


In [81]:
## cluster 2
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Wine Shop,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Ramen Restaurant
2,St. James Town,Coffee Shop,Cosmetics Shop,Hotel,Clothing Store,Cocktail Bar,Italian Restaurant,Café,Seafood Restaurant,American Restaurant,Restaurant
3,The Beaches,Health Food Store,Trail,Pub,Asian Restaurant,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Cheese Shop,Restaurant,Breakfast Spot
5,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Pizza Place,Plaza,Bubble Tea Shop,Cosmetics Shop,Furniture / Home Store,Bookstore
6,Christie,Café,Grocery Store,Italian Restaurant,Coffee Shop,Baby Store,Athletics & Sports,Candy Store,Playground,Event Space,Dumpling Restaurant
7,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,American Restaurant,Salad Place,Steakhouse,Asian Restaurant
8,"Dufferin, Dovercourt Village",Grocery Store,Park,Bakery,Pet Store,Pharmacy,Gym,Pizza Place,Middle Eastern Restaurant,Café,Smoke Shop
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Hotel,Japanese Restaurant,Plaza,Theater,Deli / Bodega,Park,Boat or Ferry,Electronics Store,Hotel Bar


In [82]:
## cluster 3
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,Park,Wings Joint,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


In [83]:
## cluster 4
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Gym,Playground,Tennis Court,Wings Joint,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [84]:
## cluster 5
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Home Service,Fast Food Restaurant,Wings Joint,Dry Cleaner,Fish & Chips Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
